In [9]:
import pandas as pd
import numpy as np
import time
import re
import os
import json
from random import random
from selenium.webdriver import Chrome
from urllib.request import urlretrieve
from uuid import uuid4
import boto3
from io import BytesIO
from PIL import Image
import tensorflow as tf

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib.request

In [10]:
def sign_in(browser):
    
    id="" # 입력
    pwd="" # 입력

    time.sleep(2)

    input_id=browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[2]/div/label/input')
    input_pwd=browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')

    input_id.send_keys(id)
    input_pwd.send_keys(pwd)

    time.sleep(2)

    browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]').click()

    time.sleep(5)

    browser.find_element_by_xpath('/html/body/div[4]/div/div/div[3]/button[2]').click()

    time.sleep(2)

def get_likes(browser):
    """Return the number of likes over 30"""
    
    like=0
    
    try:
        likes_html = browser.find_elements_by_css_selector("button")
        
        for like_html in likes_html:
            if "좋아요" in like_html.text:
                likes = re.findall("[\d*,*]*\d+", like_html.text)
                if len(likes) > 0:
                    for num in likes:
                        num = num.replace(",","")
                        if int(num) > 30:
                            like = int(num)
        return(like)
        
    except Exception as ex:
        print("get_likes 에러발생", ex)
        return like

def get_hashtags(browser):
    """Return a list of hashtags found in all post's comments"""
    
    all_hashtags = []
    
    try:
        plus = browser.find_elements_by_class_name("dCJp8")
        while(len(plus)!= 0):
            plus[0].send_keys("\n")
            plus = browser.find_elements_by_class_name("dCJp8")
            time.sleep(1)
        
        
        comments = browser.find_elements_by_class_name("EtaWk")
        for comment in comments:
            hashtags = re.findall("#[ㄱ-ㅎㅏ-ㅣ가-힣]+", comment.text)
            if len(hashtags) > 0:
                all_hashtags.extend(hashtags)
        return list(set(all_hashtags))
    
    except Exception as ex:
        print("get_hashtags 에러발생", ex)
        return list(set(all_hashtags))
    

def get_image(hashtag, browser):
    """Download image from given url and return it's name"""
    
    name = "NaN"
    
    try:
        img = browser.find_elements_by_class_name("KL4Bh")
        img_url = img[-1].find_element_by_css_selector("article img").get_attribute("src")

        uuid =uuid4()
        urlretrieve(img_url, f"data/{hashtag}/{uuid}.jpg")
        name = f"{uuid}.jpg"
        return name
    
    except Exception as ex:
        print("get_image 에러발생", ex)
        return name

def scrape_data(hashtags, n, delay=5):
    """
    Download n images and return a dictionary with their metadata.
    """
    browser = Chrome("C:/driver/chromedriver.exe") # 입력
    
    browser.get("https://www.instagram.com/accounts/login/?hl=ko&source=auth_switcher")
    sign_in(browser)

    
     
    for hashtag in hashtags: 
        url = f"https://www.instagram.com/explore/tags/{hashtag}/"
        browser.get(url)
        time.sleep(1 + (random() * 5))
        post = "https://www.instagram.com/p/"

            
        a = browser.find_elements_by_class_name("_9AhH0")[0]
        browser.execute_script("arguments[0].click();",a)
  
        time.sleep(1 + (random() * 5))
    
        posts=[]
        likes=[]
        hashtags=[]
        image_local_name=[]
         
        try:
            os.mkdir(f"data/{hashtag}")
        except OSError:
            pass # We probably tried to make something that already exists

        while len(likes) < n:
            like = get_likes(browser)
            time.sleep(random() * delay)
            if(like != 0):
                likes.append(like)
                hashtags.append(get_hashtags(browser))
                time.sleep(random() * delay)
                image_local_name.append(get_image(hashtag, browser))
                time.sleep(random() * delay)
            
            next=browser.find_element_by_class_name("coreSpriteRightPaginationArrow")
            next.click()
            time.sleep(1 + (random() * 5))
         
        
        posts = [
            {"likes": likes[i], "hashtags":hashtags[i], "image_local_name":image_local_name[i], "search_hashtag": hashtag}
            for i in range(len(likes))
        ]
        
        new_hashtag_metadata = posts
        
        #NOTE TO SELF: transferred code begins here
        if os.path.exists(f"metadata/{hashtag}.json"):
            # We already have metadata for this hashtag, add to it
            with open(f"metadata/{hashtag}.json", "r") as f:
                hashtag_metadata = json.load(f)
                hashtag_metadata += new_hashtag_metadata
        else:
            # We don't have metadata for this hashtag yet, initialize it
            hashtag_metadata = new_hashtag_metadata

        with open(f"metadata/{hashtag}.json", "w") as f:
            json.dump(hashtag_metadata, f)
            

def NaN_scrape_data(hashtags, n, delay=5):
    """
    Download n images and return a dictionary with their metadata.
    """
    browser = Chrome("C:/driver/chromedriver.exe") # 입력
    
    browser.get("https://www.instagram.com/accounts/login/?hl=ko&source=auth_switcher")
    sign_in(browser)
    
    for hashtag in hashtags: 
        
        if os.path.exists(f"metadata/{hashtag}.json"):
            with open(f"metadata/{hashtag}.json", "r", encoding='utf-8') as f:
                posts = json.load(f)
        
        try:
            for post in posts:
                if len(post) != 4:
                    like = get_likes(browser)
                    time.sleep(random() * delay)
                    if(like != 0):
                        post["likes"] = like
                        post["hashtags"] = get_hashtags(browser)
                        time.sleep(random() * delay)
                        post["image_local_name"] = get_image(hashtag, browser)
                        time.sleep(random() * delay)                    
                    next=browser.find_element_by_class_name("coreSpriteRightPaginationArrow")
                    next.click()
                    time.sleep(1 + (random() * 5))
            new_hashtag_metadata = posts
            print("NaN_scrape_data 성공")
        except Exception as ex:
            new_hashtag_metadata = posts
            print("NaN_scrape_data 에러발생", ex)

        with open(f"metadata/{hashtag}.json", "w") as f:
            json.dump(new_hashtag_metadata, f)

In [11]:
# Make sure our data and metadata folders exist before we start scraping
folder_names = ["data", "metadata"]
for folder_name in folder_names:
    try:
        os.mkdir(folder_name)
    except OSError:
        print(f"Folder '{folder_name}' already exists.")

Folder 'data' already exists.
Folder 'metadata' already exists.


In [12]:
# EXAMPLE:
# hashtags = ["travel", "food", "animals", "selfie", "cars", "fitness", "babies", "wedding", "nature", "architecture"]

# Your own hashtags here:
hashtags = ["음식스타그램", "육아", "아기스타그램"]

# How many hashtags to scrape:
num_to_scrape = 100

In [13]:
scrape_data(hashtags, num_to_scrape, delay=10)

In [6]:
taste = pd.read_json("metadata/맛스타그램.json", encoding='utf-8')
taste

,likes,hashtags,image_local_name,search_hashtag
0,363,"[#울산, #홍대, #선팔하면맞팔, #전신샷, #맞팔환영, #데일리, #맛스타그램,...",e037441f-155c-4a5e-98f6-ee9ce2e91449.jpg,맛스타그램
1,158,[],0c5aa177-ae8b-4286-9ca5-6ba465ebe0e1.jpg,맛스타그램
2,368,"[#킥킥킥, #새벽이고, #저녁이고, #아침이고, #감사합니다, #언제든연락주세요,...",ee052a36-e622-41cc-b075-0d7c5f6313a4.jpg,맛스타그램
3,272,"[#김볶, #보끔밥, #맛있다그램, #음식그램, #먹스타그램, #대존맛, #김치볶음...",74124da4-efea-4119-bd6d-eb6683d18e6b.jpg,맛스타그램
4,105,"[#짬뽕스타그램, #배달짬뽕, #고추짬뽕, #양평반점, #양평짬뽕, #맛스타그램]",b9f11116-d20a-467e-9a3e-a95e54e81df2.jpg,맛스타그램
5,149,"[#김포, #인천, #술스타그램, #맛스타그램, #서울]",941112ec-0194-4cb7-b259-385501f49d87.jpg,맛스타그램
6,367,"[#솥밥, #서울맛집, #도꼭지, #카페투어, #효니, #대흥역]",202ce85a-4bb7-4d77-af10-32ee0a89abf5.jpg,맛스타그램
7,337,"[#커플, #카페, #삼겨살, #커플스타그램, #맛스타그램, #돼지]",d209a876-5f17-4f50-98fa-2201c8c6f9df.jpg,맛스타그램
8,272,[],8a355c65-ae7e-49d1-817c-4b22d0e14db9.jpg,맛스타그램
9,33,[],8176c3c5-adef-4c2f-a885-663f9f8fbe07.jpg,맛스타그램


In [7]:
sum(taste["image_local_name"]=="NaN")

0

In [14]:
food = pd.read_json("metadata/음식스타그램.json", encoding='utf-8')
food

,likes,hashtags,image_local_name,search_hashtag
0,198,"[#선팔하면맞팔가요, #선팔하면맞팔, #먹스타, #일상소통, #맞팔환영, #선팔맞팔...",5cea38ab-c042-4260-be4b-63614a0b2738.jpg,음식스타그램
1,231,[],6250f3d6-1131-4abc-b7ee-15c2222e1c18.jpg,음식스타그램
2,208,"[#먹부림, #치킨추천, #음식스타그램, #먹스타맞팔, #치맥, #먹스타그램, #먹...",b1014e26-e712-4b56-9c6f-e27feef9ac74.jpg,음식스타그램
3,208,"[#맛있다그램, #요리하는여자, #푸드스타그램, #홈쿡, #인스타푸드, #집밥, #...",5ed2389a-05bb-4a22-8e86-b99366c37367.jpg,음식스타그램
4,177,"[#음식스타그램, #초밥, #일상, #먹스타그램, #먹방, #동성로, #대구, #맛...",4a171227-205c-4be5-983a-81048c2f5ace.jpg,음식스타그램
5,277,"[#오늘뭐먹지, #맛있다그램, #먹스타, #참치, #먹방그램, #맛집, #맛스타그램...",e94d5d10-7067-4207-bf0e-b40b450a86cf.jpg,음식스타그램
6,139,[#조선짬뽕],0e257144-00ef-433a-8752-e8061442a4be.jpg,음식스타그램
7,106,"[#삼쏘, #껍데기, #맞팔환영, #마포역맛집, #선팔환영, #맛스타그램, #이억만...",95f5c2d7-c7ce-471f-b38e-126120b8a348.jpg,음식스타그램
8,46,"[#신혼집인테리어, #전주에코시티, #새댁일상, #신혼부부, #새댁스타그램, #집들...",7aa35779-3685-4467-bbb3-59b74f46db86.jpg,음식스타그램
9,36,"[#해물찜닭, #푸드스타그램, #선팔하면맞팔, #먹스타, #맛집, #맛스타그램, #...",f770fbaf-90c8-4c79-9401-f56405a93c45.jpg,음식스타그램


In [15]:
sum(food["image_local_name"]=="NaN")

0

In [16]:
childcare = pd.read_json("metadata/육아.json", encoding='utf-8')
childcare

,likes,hashtags,image_local_name,search_hashtag
0,139,"[#아기턱받이, #생후, #아기편집샵, #이번주도고생했다, #돗투돗, #베비승준, ...",bf8e434c-5faa-4645-bbf4-acbca71e26d8.jpg,육아
1,365,"[#루다공주, #딸스타그램]",4b686200-e942-427c-8f19-b1468d9b4a55.jpg,육아
2,2238,"[#신라면좋아함, #매운새우깡좋아함, #땡초러버, #내놔, #생양파좋아함, #고추장...",aad9388f-7b4a-4d74-be2d-ee0dee0e3608.jpg,육아
3,191,"[#아기방꾸미기, #주말, #백일촬영룩, #육아, #육아템, #이쁜아기옷, #아들스...",6183803e-be08-4860-9ec6-e78d49ad1132.jpg,육아
4,114,[#한미완전두유],183caa6c-3d39-4577-8175-e77c2fb23796.jpg,육아
5,135,"[#딸맘, #아기스타그램, #육아일상, #도치맘, #육아, #황금돼지띠맘, #황금돼...",5f8337d8-f458-4ddd-ac2c-dce200cb6ce9.jpg,육아
6,378,"[#신혼일기, #팔로워늘리기, #럽스타그램, #선팔하면맞팔, #맞팔환영, #육아, ...",2e256d45-2069-4734-b3b4-2260bb4773d9.jpg,육아
7,111,[],b22b755d-7455-4e1b-869a-75aa37d94515.jpg,육아
8,2173,"[#초, #삼남매, #아기, #초딩, #포동포동, #남매, #럽스타그램, #사랑둥이...",57cd7d1e-e6ba-42dd-b538-62743b70c4a2.jpg,육아
9,89,"[#키즈홍삼, #엘더베리시럽, #아기홍삼, #어린이홍삼, #아기홍삼추천, #어린이홍...",edd9501a-b99f-4028-997a-6d4e4d3b864a.jpg,육아


In [17]:
sum(childcare["image_local_name"]=="NaN")

0

In [18]:
babies = pd.read_json("metadata/아기스타그램.json", encoding='utf-8')
babies

,likes,hashtags,image_local_name,search_hashtag
0,306,[],e4c8a766-04cb-4bfd-ae98-e912a12d6266.jpg,아기스타그램
1,142,"[#아기턱받이, #생후, #아기편집샵, #이번주도고생했다, #돗투돗, #베비승준, ...",3365a93e-775c-4608-b13f-8fea8d83a2bc.jpg,아기스타그램
2,113,"[#메이플림, #북한강]",4b3d6291-4793-461e-a0c9-29505179d8e1.jpg,아기스타그램
3,141,"[#딸맘, #아기스타그램, #육아일상, #도치맘, #육아, #황금돼지띠맘, #황금돼...",c24db5e2-e42c-4060-aa6d-ad671e184d16.jpg,아기스타그램
4,119,[],f4e73ede-0dc5-4953-b4f5-695e9b23c7ef.jpg,아기스타그램
5,245,"[#공동육아, #육아공감, #육아정보, #육아피드, #성장기록, #낮잠, #육아맞팔...",b88ff200-26ae-4ae4-8582-d0baeda189db.jpg,아기스타그램
6,1040,"[#육아용품, #육아소통환영, #선팔하면맞팔, #일상소통, #목포맛집, #육아템, ...",9826a9d6-6a2b-4392-a7ee-1b4d4cf0c16e.jpg,아기스타그램
7,172,[],aada3cf7-924f-43d6-9a9e-74653dcaba9f.jpg,아기스타그램
8,166,"[#로또, #황금돼지띠, #왕자]",be83518f-cd5c-4812-a350-6af0ec52563e.jpg,아기스타그램
9,108,"[#또피터지게싸우겠지, #윤빵떡그램, #이러고아침에눈뜨면, #새벽에감수성터지네]",a7dff0dd-f1f7-4a29-8f92-3d7226ffa09a.jpg,아기스타그램


In [19]:
sum(babies["image_local_name"]=="NaN")

0

In [ ]:
#NaN_scrape_data(hashtags, num_to_scrape, delay=10)